In [1]:
import findspark
findspark.init()

In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lineage').getOrCreate()

In [10]:
df1 = spark.read.option("multiLine","true").json("D://TechnoAvengers//Sparkp-New//movies.json")

In [11]:
df1.printSchema()

root
 |-- _id: string (nullable = true)
 |-- description: string (nullable = true)
 |-- genre: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- name: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- year: long (nullable = true)



In [30]:
from pyspark.sql.functions import col
df2 = df1.filter("rating >4")
df3 = df2.filter(col("description").contains("romantic"))

In [31]:
df3.rdd.toDebugString()


b'(1) MapPartitionsRDD[14] at javaToPython at NativeMethodAccessorImpl.java:0 []\n |  MapPartitionsRDD[13] at javaToPython at NativeMethodAccessorImpl.java:0 []\n |  SQLExecutionRDD[12] at javaToPython at NativeMethodAccessorImpl.java:0 []\n |  MapPartitionsRDD[11] at javaToPython at NativeMethodAccessorImpl.java:0 []\n |  MapPartitionsRDD[10] at javaToPython at NativeMethodAccessorImpl.java:0 []\n |  FileScanRDD[9] at javaToPython at NativeMethodAccessorImpl.java:0 []'

In [26]:
df3.explain(mode="extended")

== Parsed Logical Plan ==
'Filter Contains('description, romantic)
+- Filter (rating#4 > cast(4 as double))
   +- Relation[_id#0,description#1,genre#2,name#3,rating#4,year#5L] json

== Analyzed Logical Plan ==
_id: string, description: string, genre: array<string>, name: string, rating: double, year: bigint
Filter Contains(description#1, romantic)
+- Filter (rating#4 > cast(4 as double))
   +- Relation[_id#0,description#1,genre#2,name#3,rating#4,year#5L] json

== Optimized Logical Plan ==
Filter (((isnotnull(rating#4) AND isnotnull(description#1)) AND (rating#4 > 4.0)) AND Contains(description#1, romantic))
+- Relation[_id#0,description#1,genre#2,name#3,rating#4,year#5L] json

== Physical Plan ==
*(1) Project [_id#0, description#1, genre#2, name#3, rating#4, year#5L]
+- *(1) Filter (((isnotnull(rating#4) AND isnotnull(description#1)) AND (rating#4 > 4.0)) AND Contains(description#1, romantic))
   +- FileScan json [_id#0,description#1,genre#2,name#3,rating#4,year#5L] Batched: false, Dat

In [33]:
df3.explain(mode="codegen")

Found 1 WholeStageCodegen subtrees.
== Subtree 1 / 1 (maxMethodCodeSize:607; maxConstantPoolSize:174(0.27% used); numInnerClasses:0) ==
*(1) Project [_id#0, description#1, genre#2, name#3, rating#4, year#5L]
+- *(1) Filter (((isnotnull(rating#4) AND isnotnull(description#1)) AND (rating#4 > 4.0)) AND Contains(description#1, romantic))
   +- FileScan json [_id#0,description#1,genre#2,name#3,rating#4,year#5L] Batched: false, DataFilters: [isnotnull(rating#4), isnotnull(description#1), (rating#4 > 4.0), Contains(description#1, romantic)], Format: JSON, Location: InMemoryFileIndex[file:/D:/TechnoAvengers/Sparkp-New/movies.json], PartitionFilters: [], PushedFilters: [IsNotNull(rating), IsNotNull(description), GreaterThan(rating,4.0), StringContains(description,r..., ReadSchema: struct<_id:string,description:string,genre:array<string>,name:string,rating:double,year:bigint>

Generated code:
/* 001 */ public Object generate(Object[] references) {
/* 002 */   return new GeneratedIteratorForCode

In [35]:
df3.explain(mode="formatted")

== Physical Plan ==
* Project (3)
+- * Filter (2)
   +- Scan json  (1)


(1) Scan json 
Output [6]: [_id#0, description#1, genre#2, name#3, rating#4, year#5L]
Batched: false
Location: InMemoryFileIndex [file:/D:/TechnoAvengers/Sparkp-New/movies.json]
PushedFilters: [IsNotNull(rating), IsNotNull(description), GreaterThan(rating,4.0), StringContains(description,romantic)]
ReadSchema: struct<_id:string,description:string,genre:array<string>,name:string,rating:double,year:bigint>

(2) Filter [codegen id : 1]
Input [6]: [_id#0, description#1, genre#2, name#3, rating#4, year#5L]
Condition : (((isnotnull(rating#4) AND isnotnull(description#1)) AND (rating#4 > 4.0)) AND Contains(description#1, romantic))

(3) Project [codegen id : 1]
Output [6]: [_id#0, description#1, genre#2, name#3, rating#4, year#5L]
Input [6]: [_id#0, description#1, genre#2, name#3, rating#4, year#5L]


